## AI 기반 네트워크 이상 탐지 모델링

### 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings

# 불필요한 경고 메시지 무시 설정
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

### 1. 파일 경로 설정

In [ ]:
RAW_FILE_PATH = "../../data/traffic/CSE-CIC-IDS2018TrafficForML_CICFlowMeter_merged.csv"
SAMPLED_FILE_PATH = "../../data/traffic/CSE-CIC-IDS2018_sampled_1_3.csv"
MODEL_PATH = "../../model/traffic/traffic_model.joblib"

### 2. (최초 1회 실행) 원본 데이터 샘플링
- **메모리 오류 방지**: 대용량 원본 파일을 작은 조각(청크)으로 나누어 읽고 처리합니다.

In [7]:
# 샘플링된 파일이 이미 존재하면 이 단계는 건너뜁니다.
if not os.path.exists(SAMPLED_FILE_PATH):
    print(f"샘플링 파일이 없어 원본 데이터 샘플링을 시작합니다: {RAW_FILE_PATH}")

    # 대용량 파일을 나눠 읽기 위한 청크 크기 설정
    chunk_size = 1_000_000
    sampled_chunks = []

    try:
        # chunksize 옵션을 사용하여 파일을 조각내어 읽는 객체 생성
        reader = pd.read_csv(RAW_FILE_PATH, chunksize=chunk_size, low_memory=False)
        
        print("대용량 파일 청크 단위 샘플링 시작...")
        # 각 청크를 순회하며 처리
        for i, chunk in enumerate(reader):
            print(f"  - {i+1}번째 청크 처리 및 샘플링 중...")
            
            # 각 청크 내에서 1/3 비율로 계층적 샘플링 수행
            _, chunk_sample = train_test_split(
                chunk,
                test_size=1/3,
                random_state=0
            )
            sampled_chunks.append(chunk_sample)
        
        # 샘플링된 모든 청크들을 하나의 데이터프레임으로 병합
        data_sample = pd.concat(sampled_chunks, ignore_index=True)
        
        print("\n샘플링 완료.")
        print(f"샘플링 후 데이터 형태: {data_sample.shape}")
        
        # 최종 샘플링된 데이터 저장
        os.makedirs(os.path.dirname(SAMPLED_FILE_PATH), exist_ok=True)
        data_sample.to_csv(SAMPLED_FILE_PATH, index=False)
        print(f"샘플링된 데이터가 '{SAMPLED_FILE_PATH}'에 저장되었습니다.")

    except Exception as e:
        print(f"\n파일 처리 중 오류가 발생했습니다: {e}")
        print("파일 경로가 정확한지, 파일에 손상이 없는지 확인해 주세요.")

else:
    print(f"'{os.path.basename(SAMPLED_FILE_PATH)}' 파일이 이미 존재하여 샘플링을 건너뜁니다.")

샘플링 파일이 없어 원본 데이터 샘플링을 시작합니다: ../../data/traffic/CSE-CIC-IDS2018TrafficForML_CICFlowMeter_merged.csv
대용량 파일 청크 단위 샘플링 시작...
  - 1번째 청크 처리 및 샘플링 중...
  - 2번째 청크 처리 및 샘플링 중...
  - 3번째 청크 처리 및 샘플링 중...
  - 4번째 청크 처리 및 샘플링 중...
  - 5번째 청크 처리 및 샘플링 중...
  - 6번째 청크 처리 및 샘플링 중...
  - 7번째 청크 처리 및 샘플링 중...
  - 8번째 청크 처리 및 샘플링 중...
  - 9번째 청크 처리 및 샘플링 중...
  - 10번째 청크 처리 및 샘플링 중...
  - 11번째 청크 처리 및 샘플링 중...
  - 12번째 청크 처리 및 샘플링 중...
  - 13번째 청크 처리 및 샘플링 중...
  - 14번째 청크 처리 및 샘플링 중...
  - 15번째 청크 처리 및 샘플링 중...
  - 16번째 청크 처리 및 샘플링 중...
  - 17번째 청크 처리 및 샘플링 중...

샘플링 완료.
샘플링 후 데이터 형태: (5411012, 80)
샘플링된 데이터가 '../../data/traffic/CSE-CIC-IDS2018_sampled_1_3.csv'에 저장되었습니다.


### 3. 데이터 로드, 전처리 및 학습 준비

In [8]:
print("\n--- 데이터 처리 시작 ---")


# SELECTED_FEATURES_ORIGINAL = [
#     # --- Packetbeat에서 직접 수집 가능한 필드 ---
#     'Dst Port',
#     'Protocol',
#     'Flow Duration',
#     'Tot Fwd Pkts',
#     'Tot Bwd Pkts',
#     'TotLen Fwd Pkts',
#     'TotLen Bwd Pkts',
    
#     # --- 간단한 계산으로 생성 가능한 필드 ---
#     'Flow Byts/s',
#     'Flow Pkts/s',
#     'Fwd Pkts/s',
#     'Bwd Pkts/s',
#     'Down/Up Ratio',
#     'Pkt Size Avg',
#     'Fwd Seg Size Avg', # 'Fwd Pkt Len Mean'의 근사치로 사용 가능
#     'Bwd Seg Size Avg', # 'Bwd Pkt Len Mean'의 근사치로 사용 가능
# ]
SELECTED_FEATURES_ORIGINAL = [
    'Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
    'TotLen Fwd Pkts', 'TotLen Bwd Pkts',
    'Flow Byts/s', 'Flow Pkts/s', 'Bwd IAT Tot', 'FIN Flag Cnt', 'RST Flag Cnt',
    'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'Down/Up Ratio',
    'Pkt Size Avg', 'Fwd Seg Size Avg', 'Fwd Pkt Len Mean', 'Bwd Seg Size Avg', 'Bwd Pkt Len Mean'
]

# 사용할 컬럼('Label' 포함)만 지정하여 로드
COLS_TO_LOAD = SELECTED_FEATURES_ORIGINAL + ['Label']
data = pd.read_csv(SAMPLED_FILE_PATH, usecols=COLS_TO_LOAD, low_memory=False)
print(f"데이터 로드 완료. 형태: {data.shape}")

# 컬럼명의 공백을 언더스코어(_)로, 슬래시(/)를 _per_로 변경
data.columns = data.columns.str.replace(' ', '_').str.replace('/', '_per_')
print("컬럼명 변경 완료 (예: 'Dst Port' -> 'Dst_Port').")

# 변경된 컬럼명에 맞춰 피처 목록도 업데이트
SELECTED_FEATURES_RENAMED = [col.replace(' ', '_').replace('/', '_per_') for col in SELECTED_FEATURES_ORIGINAL]

# 레이블 통합
label_mapping = {
    'Benign': 'Benign', 'Bot': 'Bot', 'Infilteration': 'Bot',
    'DoS attacks-SlowHTTPTest': 'DDoS', 'DoS attacks-Hulk': 'DDoS',
    'DDoS attacks-LOIC-HTTP': 'DDoS', 'DoS attacks-GoldenEye': 'DDoS',
    'DoS attacks-Slowloris': 'DDoS', 'DDOS attack-LOIC-UDP': 'DDoS',
    'DDOS attack-HOIC': 'DDoS', 'Brute Force -Web': 'Brute Force',
    'Brute Force -XSS': 'Brute Force', 'FTP-BruteForce': 'Brute Force',
    'SSH-Bruteforce': 'Brute Force', 'SQL Injection': 'SQL Injection', 'XSS': 'XSS'
}
data['Label'] = data['Label'].map(label_mapping)
filtered_data = data.dropna(subset=['Label']).copy()

# 데이터 분할 (Feature & Label) - 변경된 컬럼명 사용
X = filtered_data[SELECTED_FEATURES_RENAMED]
y = filtered_data['Label']

# 데이터 분할 (Train & Validation)
x_train, x_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y
)
print(f"훈련/검증 데이터 분할 완료. 훈련 데이터 형태: {x_train.shape}")

# 모든 피처 컬럼을 강제로 숫자 타입으로 변환하고, 변환 불가 값은 결측치(NaN)로 처리
for col in x_train.columns:
    x_train[col] = pd.to_numeric(x_train[col], errors='coerce')
    x_val[col] = pd.to_numeric(x_val[col], errors='coerce')
    
x_train.replace([np.inf, -np.inf], np.nan, inplace=True)
x_val.replace([np.inf, -np.inf], np.nan, inplace=True)


--- 데이터 처리 시작 ---
데이터 로드 완료. 형태: (5411012, 22)
컬럼명 변경 완료 (예: 'Dst Port' -> 'Dst_Port').
훈련/검증 데이터 분할 완료. 훈련 데이터 형태: (4328794, 21)


In [9]:
# 결측치 처리 (중앙값으로 대체)
imputer = SimpleImputer(strategy='median')
x_train = pd.DataFrame(imputer.fit_transform(x_train), columns=x_train.columns)
x_val = pd.DataFrame(imputer.transform(x_val), columns=x_val.columns)

# 스케일링
scaler = RobustScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

# 인코딩
le = LabelEncoder()
y_train_final = le.fit_transform(y_train)
y_val_final = le.transform(y_val)

### 4. 모델 학습 및 평가

##### 머신러닝

In [12]:
# --- 1. XGBoost 모델 평가 ---
xgb = XGBClassifier(
    random_state=0,
    n_jobs=-1,
    eval_metric='mlogloss',
    use_label_encoder=False
)
xgb.fit(x_train_scaled, y_train_final)
pred_xgb = xgb.predict(x_val_scaled)

accuracy_xgb = accuracy_score(y_val_final, pred_xgb)
precision_xgb = precision_score(y_val_final, pred_xgb, average='macro', zero_division=0)
recall_xgb = recall_score(y_val_final, pred_xgb, average='macro', zero_division=0)
f1_xgb = f1_score(y_val_final, pred_xgb, average='macro', zero_division=0)

print("========== XGBoost 모델 평가 결과 ==========")
print(f"정확도 (Accuracy): {accuracy_xgb:.4f}")
print(f"정밀도 (Precision): {precision_xgb:.4f}")
print(f"재현율 (Recall): {recall_xgb:.4f}")
print(f"F1 스코어 (F1-Score): {f1_xgb:.4f}")
print("============================================")


# --- 2. LightGBM 모델 평가 ---
lgbm = LGBMClassifier(random_state=0, n_jobs=-1)
lgbm.fit(x_train_scaled, y_train_final)
pred_lgbm = lgbm.predict(x_val_scaled)

accuracy_lgbm = accuracy_score(y_val_final, pred_lgbm)
precision_lgbm = precision_score(y_val_final, pred_lgbm, average='macro', zero_division=0)
recall_lgbm = recall_score(y_val_final, pred_lgbm, average='macro', zero_division=0)
f1_lgbm = f1_score(y_val_final, pred_lgbm, average='macro', zero_division=0)

print("\n========== LightGBM 모델 평가 결과 ==========")
print(f"정확도 (Accuracy): {accuracy_lgbm:.4f}")
print(f"정밀도 (Precision): {precision_lgbm:.4f}")
print(f"재현율 (Recall): {recall_lgbm:.4f}")
print(f"F1 스코어 (F1-Score): {f1_lgbm:.4f}")
print("=============================================")


# --- 3. CatBoost 모델 평가 ---
cat = CatBoostClassifier(random_state=0, verbose=0)
cat.fit(x_train_scaled, y_train_final)
pred_cat = cat.predict(x_val_scaled)

accuracy_cat = accuracy_score(y_val_final, pred_cat)
precision_cat = precision_score(y_val_final, pred_cat, average='macro', zero_division=0)
recall_cat = recall_score(y_val_final, pred_cat, average='macro', zero_division=0)
f1_cat = f1_score(y_val_final, pred_cat, average='macro', zero_division=0)

print("\n========== CatBoost 모델 평가 결과 ==========")
print(f"정확도 (Accuracy): {accuracy_cat:.4f}")
print(f"정밀도 (Precision): {precision_cat:.4f}")
print(f"재현율 (Recall): {recall_cat:.4f}")
print(f"F1 스코어 (F1-Score): {f1_cat:.4f}")
print("=============================================")

========== XGBoost 모델 평가 결과 ==========
정확도 (Accuracy): 0.9806
정밀도 (Precision): 0.9567
재현율 (Recall): 0.7378
F1 스코어 (F1-Score): 0.7754
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037104 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3603
[LightGBM] [Info] Number of data points in the train set: 4328794, number of used features: 21
[LightGBM] [Info] Start training from score -0.185555
[LightGBM] [Info] Start training from score -3.589912
[LightGBM] [Info] Start training from score -3.750624
[LightGBM] [Info] Start training from score -2.134989
[LightGBM] [Info] Start training from score -11.879602

========== LightGBM 모델 평가 결과 ==========
정확도 (Accuracy): 0.9686
정밀도 (Precision): 0.7251
재현율 (Recall): 0.6952
F1 스코어 (F1-Score): 0.7050

========== CatBoost 모델 평가 결과 ==========
정확도 (Accuracy): 0.9804
정밀도 (Precision): 0.9552
재현율 (Recall): 0.73

##### 딥러닝

In [ ]:
# import time
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.callbacks import EarlyStopping

# # --- 1. 데이터 준비 (딥러닝 모델용 추가 전처리) ---
# # 딥러닝 모델은 타겟(y) 데이터를 원-핫 인코딩(One-Hot Encoding)해야 합니다.
# # 예: 3개 클래스 -> [0, 1, 2] 를 [[1,0,0], [0,1,0], [0,0,1]] 형태로 변환
# num_classes = len(np.unique(y_train_final)) # 전체 클래스 개수
# y_train_categorical = to_categorical(y_train_final, num_classes=num_classes)
# y_val_categorical = to_categorical(y_val_final, num_classes=num_classes)

# # 피처 개수 확인
# num_features = x_train_scaled.shape[1]

# # --- 2. 딥러닝 모델(MLP) 정의 ---
# model = Sequential([
#     # 입력층: input_shape에 피처 개수를 지정
#     Dense(128, activation='relu', input_shape=(num_features,)),
#     Dropout(0.3), # 과적합 방지를 위한 드롭아웃
    
#     # 은닉층
#     Dense(64, activation='relu'),
#     Dropout(0.3),
    
#     # 출력층: 클래스 개수만큼 뉴런을 배치하고, 다중 분류를 위해 softmax 활성화 함수 사용
#     Dense(num_classes, activation='softmax')
# ])

# # --- 3. 모델 컴파일 ---
# # 학습 과정에 사용할 옵티마이저, 손실 함수, 평가 지표를 설정합니다.
# model.compile(
#     optimizer='adam', # 가장 일반적으로 사용되는 옵티마이저
#     loss='categorical_crossentropy', # 다중 분류의 표준 손실 함수
#     metrics=['accuracy'] # 학습 과정에서 정확도를 모니터링
# )

# # 모델 구조 요약 출력
# model.summary()

# # --- 4. 모델 훈련 ---
# # 조기 종료(Early Stopping) 설정: 검증 손실(val_loss)이 5번 이상 개선되지 않으면 학습을 중단
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# print("\n딥러닝 모델 훈련 시작...")
# start_time = time.time()

# history = model.fit(
#     x_train_scaled, 
#     y_train_categorical,
#     epochs=100, # 최대 100번 학습 반복
#     batch_size=256, # 한 번에 256개씩 데이터를 묶어 학습
#     validation_data=(x_val_scaled, y_val_categorical),
#     callbacks=[early_stopping],
#     verbose=1 # 학습 과정 출력
# )

# end_time = time.time()
# print("모델 훈련 완료.")

# # --- 5. 최종 예측 및 평가 ---
# # 딥러닝 모델의 predict 결과는 각 클래스에 대한 확률값이므로, 가장 높은 확률의 인덱스를 찾아야 함
# pred_probs = model.predict(x_val_scaled)
# pred_labels = np.argmax(pred_probs, axis=1)

# # 최종 평가 지표 계산 (평가 시에는 원본 y_val_final 사용)
# accuracy = accuracy_score(y_val_final, pred_labels)
# precision = precision_score(y_val_final, pred_labels, average='macro', zero_division=0)
# recall = recall_score(y_val_final, pred_labels, average='macro', zero_division=0)
# f1 = f1_score(y_val_final, pred_labels, average='macro', zero_division=0)

# # 최종 결과 출력
# print(f"\n학습 및 예측 수행 시간: {end_time - start_time:.2f}초")
# print("========== 딥러닝 모델 평가 결과 ==========")
# print(f"정확도 (Accuracy): {accuracy:.4f}")
# print(f"정밀도 (Precision): {precision:.4f}")
# print(f"재현율 (Recall): {recall:.4f}")
# print(f"F1 스코어 (F1-Score): {f1:.4f}")
# print("=========================================")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2816      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 11,397
Trainable params: 11,397
Non-trainable params: 0
_________________________________________________________________

딥러닝 모델 훈련 시작...
Epoch 1/100
16910/16910 [==========

### 5. 훈련된 모델 저장

In [13]:
# 모델을 저장할 폴더가 없으면 생성
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

# 1. 모델 저장
joblib.dump(xgb, MODEL_PATH)
print(f"학습된 최적 모델이 '{MODEL_PATH}'에 저장되었습니다.")

# 2. 결측치 대체기(Imputer) 저장  <- 이 부분 추가!
IMPUTER_PATH = os.path.join(os.path.dirname(MODEL_PATH), "imputer.joblib")
joblib.dump(imputer, IMPUTER_PATH)
print(f"결측치 대체기가 '{IMPUTER_PATH}'에 저장되었습니다.")

# 3. 스케일러(Scaler) 저장
SCALER_PATH = os.path.join(os.path.dirname(MODEL_PATH), "scaler.joblib")
joblib.dump(scaler, SCALER_PATH)
print(f"스케일러가 '{SCALER_PATH}'에 저장되었습니다.")

# 4. 레이블 인코더(LabelEncoder) 저장
ENCODER_PATH = os.path.join(os.path.dirname(MODEL_PATH), "label_encoder.joblib")
joblib.dump(le, ENCODER_PATH)
print(f"레이블 인코더가 '{ENCODER_PATH}'에 저장되었습니다.")

학습된 최적 모델이 '../../model/traffic/traffic_model.joblib'에 저장되었습니다.
결측치 대체기가 '../../model/traffic\imputer.joblib'에 저장되었습니다.
스케일러가 '../../model/traffic\scaler.joblib'에 저장되었습니다.
레이블 인코더가 '../../model/traffic\label_encoder.joblib'에 저장되었습니다.
